# Introduction

STREAM is a trajectory inference method that can accurately reconstruct complex developmental trajectories. It also provides informative and intuitive visualizations to recover and highlight important genes that define subpopulations and cell types. STREAM takes as input a single-cell gene expression (or epigenomic profile) matrix and approximates the data in three or more dimensions with a structure called the principal graph, a set of curves that naturally describe the cells’ pseudotime, trajectories, and branching points.


<img src="https://github.com/pinellolab/STREAM/raw/stream_python2/STREAM/static/images/Figure1.png">
<p><span style="padding-left:20px">&nbsp;&nbsp;</span><b>Figure:</b> Stream pipeline overview on single-cell RNA-Seq data from the mouse hematopoietic system.  Image from Figure 1 of <a href="https://www.nature.com/articles/s41467-019-09670-4">Chen et al, 2019</a>.</p>


STREAM first identifies informative features such as variable genes or top principal components. Using these features, cells are then projected to a lower dimensional space using a non-linear dimensionality reduction method called Modified Locally Linear Embedding (MLLE), which preserves distances within local neighborhoods. In the MLLE embedding, STREAM infers cellular trajectories using an Elastic Principal Graph implementation called ElPiGraph. ElPiGraph is a completely redesigned algorithm for the previously introduced elastic principal graph optimization based on the use of elastic matrix Laplacian, trimmed mean square error, explicit control of topological complexity and scalability to millions of points on an ordinary laptop. In STREAM, the ElPiGraph was further developed to integrate a new heuristic graph structure seeding to learn principal graphs in high dimensions with several problem-specific topological graph grammar rules optimized for single-cell trajectory inference.

Datasets can be downloaded from https://www.dropbox.com/sh/dfqht1ob89ku99d/AACI5ZW3aRuq9MhBfSNS_1O_a?dl=0  
Ref: Nestorowa, S. et al. A single-cell resolution map of mouse hematopoietic stem and progenitor cell differentiation. Blood 128, e20-31 (2016).

Additional tutorials using STREAM in non-GenePattern notebooks are available in the <a href="https://github.com/pinellolab/STREAM/#tutorial">STREAM GitHub Repository</a>.

## Login
 
**Instructions are given in blue boxes, such as with the one below.**

<div class="alert alert-info"><h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Sign in to GenePattern by clicking the login button or entering your username and password into the form below.</div>
</p>

Make sure to login here before proceeding any farther in the notebook. The user interface components and analysis outputs won't appear until you are logged in.

In [29]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

## Loading Notebook Functions

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Run the cell below to load functions that will be used later in this notebook.
</div>

In [15]:
from IPython.core.display import display, HTML, display_html
from IPython.display import Image

%matplotlib inline

displayImageDescription = "Display an image within a cell of the notebook."
displayImageParamDef = {"image":{  "name":"Image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_1":{"name":"First image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_2":{"name":"Second image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "height": {"type": "number", "optional": True}, 
                        "width": {"type":"number", "optional": True}, 
                        "output_var":{"hide":True }}

def displayImage(image, height, width):
    return Image(url= image, width=width, height=height)



def displayImagePair(image_1, image_2, height, width):
  
    # display(Image(url= image_1, width=width, height=height), Image(url= image_2, width=width, height=height))
    # display(Image(url= image_2, width=width, height=height))
    tdstyle=style='vertical-align: bottom;background-color:white'
    html_str = "<table><tr><td style='"+tdstyle+"'><img src="+image_1+" height="+str(height)+" width="+str(width)+"></td><td style='"+tdstyle+"'><img src="+image_2+" height="+str(height)+" width="+str(width)+"/></td></tr></table>"
    display_html(html_str, raw=True)
    return None   


displayPdfDescription = "Display an image within a cell of the notebook."
displayPdfParamDef = {"image":{  "name":"Image file URL", "type":"file", "kinds":["pdf", "PDF"]}, 
                        "image_1":{"name":"First image file URL", "type":"file", "kinds":["pdf", "PDF"]}, 
                        "height": {"type": "number", "optional": True}, 
                        "width": {"type":"number", "optional": True}, 
                        "output_var":{"hide":True }}



class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

def displayPdf(image, height, width):
    return PDF(image,size=(width,height))


@genepattern.build_ui(name="Load Functions", description="Loads local functions used in this notebook",  collapse=False, parameters={ "output_var":{"hide":True }})
def loadFunctions():
    print("Utility functions loaded");

UIBuilder(collapse=False, description='Loads local functions used in this notebook', function_import='nbtools.…

## STREAM Sample Data
   
   For this notebook we will be using a dataset from "A single-cell resolution map of mouse hematopoietic stem and progenitor cell differentiation". This dataset can be downloaded from https://www.dropbox.com/sh/dfqht1ob89ku99d/AACI5ZW3aRuq9MhBfSNS_1O_a?dl=0   
   

This dataset contains 1656 sorted and profiled single cells including hematopoietic stem cells (HSCs), multipotent progenitors (MPPs), lymphoid multipotent progenitors (LMPPs), common myeloid progenitors (CMPs), granulocyte-monocyte progenitors (GMPs) and megakaryocyte-erythrocyte progenitors (MEPs), to study the mouse hematopoietic stem and progenitor cell differentiation processes. 
   
Ref: Nestorowa, S. et al. A single-cell resolution map of mouse hematopoietic stem and progenitor cell differentiation. Blood 128, e20-31 (2016).

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Click the 'Run' button to have links to sample data added to your notebook.
</div>

In [16]:
@genepattern.build_ui(collapse=False, parameters={
   "output_var": {
        "name": "results",
        "description": "There are the results",
        "hide": True
    }
})
def getSampleSTREAMFiles():
    # URL or file path relative to the notebook's directory
    exp_path = 'https://datasets.genepattern.org/data/module_support_files/STREAM/data_Nestorowa.tsv.gz'
    cn_path = 'https://datasets.genepattern.org/data/module_support_files/STREAM/cell_label_color.tsv.gz'
    meth_path = 'https://datasets.genepattern.org/data/module_support_files/STREAM/cell_label.tsv.gz'

    return genepattern.GPUIOutput(files=[exp_path, cn_path, meth_path])

UIBuilder(collapse=False, function_import='nbtools.tool(id="getSampleSTREAMFiles", origin="Notebook").function…

# Preprocessing

Here we will first normalize the raw gene expression values based on library size, then the gene expression values will be log transformed. The mitochondrial genes will be removed.

With this module we can filter out cells based on several cell-centric metrics, including the minimum number of genes expressed, the minimum percentage of genes expressed, and the minimum number of read counts for one cell.

We can also filter out genes based on several gene-centric metrics, including the minimum number of cells expressing one gene, the minimum percentage of cells expressing one gene, and the minimum number of read counts for one gene.

Finally, this module writes the dataset into the pkl format used by the rest of the modules in the STREAM collection.

<div class="alert alert-info">
<p class="lead"> Instructions - Preprocessing<i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.Preprocess module form below<br/>
        <UL>
            <LI>Data File - select <b>data_Nestorowa.tsv.gz</b> from the drop-down selector.</LI>
            <LI>cell label file - select <b>cell_label.tsv.gz</b> from the drop-down selector.</LI>
            <LI>cell label color select <b>file - cell_label_color.tsv.gz</b> from the drop-down selector.</LI>
            <LI>output filename - <b>nestorowa</b></LI>
            <LI>min num cells - <b>5</b></LI>
            <li>expression cutoff - <b>1</b></li>
            <li>All other parameters should be 0 or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [30]:
stream_preprocess_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00395')
stream_preprocess_job_spec = stream_preprocess_task.make_job_spec()
stream_preprocess_job_spec.set_parameter("data.file", "https://datasets.genepattern.org/data/module_support_files/STREAM/data_Nestorowa.tsv.gz")
stream_preprocess_job_spec.set_parameter("cell.label.file", "https://datasets.genepattern.org/data/module_support_files/STREAM/cell_label.tsv.gz")
stream_preprocess_job_spec.set_parameter("cell.label.color.file", "https://datasets.genepattern.org/data/module_support_files/STREAM/cell_label_color.tsv.gz")
stream_preprocess_job_spec.set_parameter("output.filename", "nestorowa")
stream_preprocess_job_spec.set_parameter("normalize", "")
stream_preprocess_job_spec.set_parameter("log.transform", "")
stream_preprocess_job_spec.set_parameter("remove.mitochondrial.genes", "")
stream_preprocess_job_spec.set_parameter("min.percent.genes", "0")
stream_preprocess_job_spec.set_parameter("min.count.genes", "0")
stream_preprocess_job_spec.set_parameter("min.percent.cells", "0")
stream_preprocess_job_spec.set_parameter("min.count.cells", "0")
stream_preprocess_job_spec.set_parameter("min.num.cells", "5")
stream_preprocess_job_spec.set_parameter("expression.cutoff", "1")
stream_preprocess_job_spec.set_parameter("job.memory", "8 Gb")
stream_preprocess_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_preprocess_job_spec.set_parameter("job.cpuCount", "2")
stream_preprocess_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_preprocess_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00395')

## Feature selection

Two types of features can be used for the downstream analysis

* variable genes
* top principal components

For transcriptomic data (single-cell RNA-seq or qPCR), the input of STREAM is a gene expression matrix, where rows represent genes, and columns represent cells. Each entry contains an adjusted gene expression value (normalized for library size and log2 transformed). 

By default the most variable genes are selected as features. Briefly, for each gene, its mean value and standard deviation are calculated across all the cells. Then a non-parametric local regression method (LOESS) is used to fit the relationship between mean and standard deviation values. Genes above the curve that diverge significantly are selected as the most variable genes. 

Alternatively, users can also perform PCA on the scaled matrix and select the top principal components based on the variance ratio elbow plot.


<div class="alert alert-info">
<p class="lead"> Instructions - Feature Selection <i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.FeatureSelection module form below<br/>
        <UL>
            <LI>Data File - select <b>nestorowa_stream_result.pkl</b> from the drop-down selector.</LI>
            <LI>output filename - <b>fs</b></LI>
            <LI>find variable genes - <b>True</b></LI>
            <li>loess fraction - <b>0.1</b></li>
            <li>percentile - <b>95</b></li>
            <li>find principal components - <b>False</b></li>
            <li>All other parameters should be left at their defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol> 
</div>

In [31]:
stream_featureselection_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00396')
stream_featureselection_job_spec = stream_featureselection_task.make_job_spec()
stream_featureselection_job_spec.set_parameter("data.file", "")
stream_featureselection_job_spec.set_parameter("loess.fraction", "0.1")
stream_featureselection_job_spec.set_parameter("percentile", "95")
stream_featureselection_job_spec.set_parameter("num.genes", "")
stream_featureselection_job_spec.set_parameter("output.filename", "fs")
stream_featureselection_job_spec.set_parameter("feature", "var_genes")
stream_featureselection_job_spec.set_parameter("find.variable.genes", "--flag_variable")
stream_featureselection_job_spec.set_parameter("find.principal.components", "")
stream_featureselection_job_spec.set_parameter("num.principal.components", "15")
stream_featureselection_job_spec.set_parameter("max.principal.components", "100")
stream_featureselection_job_spec.set_parameter("first.principal.component", "")
stream_featureselection_job_spec.set_parameter("use.precomputed", "--flag_useprecomputed")
stream_featureselection_job_spec.set_parameter("figure.height", "8")
stream_featureselection_job_spec.set_parameter("figure.width", "8")
stream_featureselection_job_spec.set_parameter("job.memory", "8 Gb")
stream_featureselection_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_featureselection_job_spec.set_parameter("job.cpuCount", "2")
stream_featureselection_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_featureselection_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00396')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Image File Url"</b> parameter to <b>"fs_variable_genes.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
    Note:  this cell may report an error stating "ERROR! Session/line number was not unique in database. History logging moved to new session".  This can be safely ignored.
</div>

In [17]:
genepattern.GPUIBuilder(displayImage, collapse=False,
                        name="Display variable genes scatterplot",
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

## Dimension Reduction

Each cell can be thought of as a vector in a multi-dimensional vector space in which each component is the expression level of a gene. Typically, even after feature selection, each cell still has hundreds of components, making it difficult to reliably assess similarity or distances between cells, a problem often referred as the curse of dimensionality. To mitigate this problem, starting from the genes selected in the previous step we project cells to a lower dimensional space using a non-linear dimensionality reduction method called Modified Locally Linear Embedding (MLLE).

Several alternative dimension reduction methods are also supported, **spectral embedding**, **umap**, and **pca**.
By default the **MLLE** method is used.

For large datasets, spectral embedding works faster than MLLE while preserving a similar compact structure to MLLE.

For large datasets, lowering the percentage of neighbor cells parameter (0.1 by default) will speed up this step.

By default we set the number of components to 3.  For biological process with simple bifurcation or linear trajectory, keeping only two components would be recommended.


<div class="alert alert-info">
<p class="lead"> Instructions - Dimension Reduction <i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.DimensionReduction module form below<br/>
        <UL>
            <LI>Data File - select the result of the Feature Selection run from the file dropdown, <b>fs_stream_result.pkl</b></LI>
            <LI>output filename - <b>dr</b></LI>
            <li>feature - <b>most variable genes</b></li>
            <LI>method - <b>Modified Locally linear embedding algoritm (MLLE)</b></LI>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [34]:
stream_dimensionreduction_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00397')
stream_dimensionreduction_job_spec = stream_dimensionreduction_task.make_job_spec()
stream_dimensionreduction_job_spec.set_parameter("data.file", "")
stream_dimensionreduction_job_spec.set_parameter("output.filename", "dr")
stream_dimensionreduction_job_spec.set_parameter("percent.neighbor.cells", ".1")
stream_dimensionreduction_job_spec.set_parameter("num.components.to.keep", "3")
stream_dimensionreduction_job_spec.set_parameter("feature", "var_genes")
stream_dimensionreduction_job_spec.set_parameter("method", "mlle")
stream_dimensionreduction_job_spec.set_parameter("num.components.to.plot", "3")
stream_dimensionreduction_job_spec.set_parameter("component.x", "0")
stream_dimensionreduction_job_spec.set_parameter("component.y", "1")
stream_dimensionreduction_job_spec.set_parameter("figure.width", "8")
stream_dimensionreduction_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_dimensionreduction_job_spec.set_parameter("figure.height", "8")
stream_dimensionreduction_job_spec.set_parameter("job.memory", "8 Gb")
stream_dimensionreduction_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_dimensionreduction_job_spec.set_parameter("job.cpuCount", "2")
stream_dimensionreduction_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_dimensionreduction_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00397')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Image File Url"</b> parameter to <b>"dr_stddev_dotplot.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
    Note:  this cell may report an error stating "ERROR! Session/line number was not unique in database. History logging moved to new session".  This can be safely ignored.
</div>

In [18]:
genepattern.GPUIBuilder(displayImage, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

### UMAP validation of cell population structure

**This step is helpful only when keeping more than two components in the step of Dimension Reduction**  
(Because in 3D or even higher-dimensional space, it is difficult to see the whole structure. UMAP or tSNE visualization can serve as an explicit summary for high-dimensional space.).

Before we begin with the Elastic Principal Graph, we will check if there is clear meaningful trajectory pattern to the data.  If there is, we will continue the downstream analysis placing the cells onto the trajectories.  If not, we would go back to previous steps to modify the parameters used to filter and prepare the data to try different settings.

To check the data, we will implement UMAP ot tSNE based on the components from the step of dimension reduction to visualize the data in 2D plane.

<div class="alert alert-info">
<p class="lead"> Instructions - Validation of population structure <i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.Plot2DVisualization module form below<br/>
        <UL>
            <LI>Data File - select the result of the Dimension Reduction run from the file dropdown, <b>dr_stream_result.pkl</b></LI>
            <LI>output filename - <b>plotted</b></LI>
            <LI>method - <b>Uniform Manifold Approximation and Projection (UMAP)</b></LI>
            <li>percent neighbor cells - <b>0.1</b></li>
            <li>color by - <b>label</b></li>
            <li>use precomputed - <b>false</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [35]:
stream_plot2dvisualization_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00406')
stream_plot2dvisualization_job_spec = stream_plot2dvisualization_task.make_job_spec()
stream_plot2dvisualization_job_spec.set_parameter("data.file", "")
stream_plot2dvisualization_job_spec.set_parameter("percent.neighbor.cells", "0.1")
stream_plot2dvisualization_job_spec.set_parameter("perplexity", "30")
stream_plot2dvisualization_job_spec.set_parameter("color.by", "label")
stream_plot2dvisualization_job_spec.set_parameter("method", "umap")
stream_plot2dvisualization_job_spec.set_parameter("use.precomputed", "")
stream_plot2dvisualization_job_spec.set_parameter("figure.width", "8")
stream_plot2dvisualization_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_plot2dvisualization_job_spec.set_parameter("figure.height", "8")
stream_plot2dvisualization_job_spec.set_parameter("output.filename", "plotted")
stream_plot2dvisualization_job_spec.set_parameter("job.memory", "8 Gb")
stream_plot2dvisualization_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_plot2dvisualization_job_spec.set_parameter("job.cpuCount", "2")
stream_plot2dvisualization_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_plot2dvisualization_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00406')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Image File Url"</b> parameter to <b>"plotted_2D_plot.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [19]:
genepattern.GPUIBuilder(displayImage, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

# Structure Learning

Elastic principal graphs are structured data approximators, consisting of vertices connected by edges. The vertices are embedded into the space of the data, minimizing the mean squared distance (MSD) to the data points, similarly to k-means. Unlike unstructured k-means, the edges connecting the vertices are used to define an elastic energy term. The elastic energy term and MSD are used to create penalties for edge stretching and bending of branches.

The STREAM.ElasticPrincipalGraph module uses the R-language ElPiGraph implementation of Elastic Principal Graphs. To find the optimal graph structure, ElPiGraph uses a topological grammar (or, graph grammar) approach. ElPiGraph is a completely redesigned algorithm for the previously introduced elastic principal graph optimization based on the use of elastic matrix Laplacian, trimmed mean square error, explicit control of topological complexity and scalability to millions of points on an ordinary laptop.

We will begin by seeding the EPG using the STREAM.SeedEPGStructure module and then with the result, we will run the ElPiGraph algorithm using the STREAM.ElasticPrincipalGraph module to generate our pseudotime trajectories.

The principal graph inference is based on a greedy optimization procedure that may lead to local minima, therefore in STREAM we proposed an initialization procedure that improves the quality of the inferred solutions and that speed up convergence. First, cells are clustered in the low-dimensional space (by default, k-means is used. Alternatively another two clustering methods including affinity propagation(**ap**) and spectral clustering(**sc**) are also available). Based on the centroids obtained, a minimum spanning tree (MST) is constructed using the Kruskal’s algorithm. The obtained tree is then used as initial tree structure for the ElPiGraph procedure.

<div class="alert alert-info">
<p class="lead"> Instructions - Seeding the Elastic Principal Graph <i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.SeedEPGStructure module form below<br/>
        <UL>
            <span><b><i>Input and Output</i></b></span>
            <LI>Data File - Select the <b>dr_stream_result.pkl</b> from the dropdown that represents the output of the DimensionReduction step above.</LI>
            <LI>Output filename - <b>seeded</b></LI>
            <span><b><i>Structure Learning</i></b></span>
            <LI>Percent neighbor cells - <b>0.1</b></LI>
            <li>Num clusters - <b>10</b></li>
            <li>Damping - <b>0.75</b></li>
            <li>Preference Percentile - <b>50</b></li>
            <li>max clusters - <b>200</b></li>
            <li>clustering - <b>K-means clustering</b></li>
            <span><b><i>Plotting</i></b></span>
            <li>num components - <b>3</b></li>
            <li>Component X - <b>0</b></li>
            <li>Component Y - <b>1</b></li>
            <li>All other parameters should be left at their default values or empty.</li>
        </UL></li>
    <li>Click the Run Button</li>
</ol>
</div>

In [37]:
stream_seedepgstructure_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00398')
stream_seedepgstructure_job_spec = stream_seedepgstructure_task.make_job_spec()
stream_seedepgstructure_job_spec.set_parameter("data.file", "")
stream_seedepgstructure_job_spec.set_parameter("num.components", "3")
stream_seedepgstructure_job_spec.set_parameter("percent.neighbor.cells", "0.1")
stream_seedepgstructure_job_spec.set_parameter("num.clusters", "10")
stream_seedepgstructure_job_spec.set_parameter("damping", "0.75")
stream_seedepgstructure_job_spec.set_parameter("preference.percentile", "50")
stream_seedepgstructure_job_spec.set_parameter("max.clusters", "200")
stream_seedepgstructure_job_spec.set_parameter("figure.width", "8")
stream_seedepgstructure_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_seedepgstructure_job_spec.set_parameter("figure.height", "8")
stream_seedepgstructure_job_spec.set_parameter("output.filename", "seeded")
stream_seedepgstructure_job_spec.set_parameter("clustering", "kmeans")
stream_seedepgstructure_job_spec.set_parameter("component.x", "0")
stream_seedepgstructure_job_spec.set_parameter("component.y", "1")
stream_seedepgstructure_job_spec.set_parameter("job.memory", "8 Gb")
stream_seedepgstructure_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_seedepgstructure_job_spec.set_parameter("job.cpuCount", "2")
stream_seedepgstructure_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_seedepgstructure_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00398')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"seeded_branches.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Set the <b>"Second image File Url"</b> parameter to <b>"seeded_branches_with_cells.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [20]:
genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…


Now that we have seeded the elastic principal graph, we will turn the module loose to learn the graph and generate the pseudotime trajectories. 

Elastic principal graphs are structured data approximators, consisting of vertices connected by edges. The vertices are embedded into the space of the data, minimizing the mean squared distance (MSD) to the data points, similarly to k-means. Unlike unstructured k-means, the edges connecting the vertices are used to define an elastic energy term. The elastic energy term and MSD are used to create penalties for edge stretching and bending of branches. To find the optimal graph structure, ElPiGraph uses a topological grammar (or, graph grammar) approach, which is described below. 

Briefly, an elastic principal graph is an undirected graph with a set of vertices V and a set of edges E. The set of vertices V is embedded in the multidimensional space by minimizing the sum of the data approximation term and the graph elastic energy. Elastic principal graphs are based on minimization of the energy potential containing three parts :

$$U = MSE + \lambda U_E + \mu U_R$$

where MSE is the mean squared error of data approximation, $U_E$ is the sum of squared edge lengths, $U_R$ is a term minimizing the deviation of the principal graph embedment from harmonicity, equation , $\lambda$, $\mu$ are coefficients of regularization.

*$epg\ trimming\ radius$*, *$epg\ lambda$*, and *$epg\ mu$* are parameters having the following meaning: *$epg\ trimming\ radius$* is the trimming radius such that points further than *$epg\ trimming\ radius$* from any node do not contribute to the optimization of the graph, *$epg\ lambda$* is the edge stretching elasticity modulo regularizing the total length of the graph edges and making their distribution close to equidistant in the multidimensional space, *$epg\ mu$* is the star bending elasticity modulo controlling the deviation of the graph stars from harmonic configurations.

<div class="alert alert-info">
<p class="lead"> Instructions - Generating the Elastic Principal Graph <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.ElasticPrincipalGraph module form below<br/>
        <UL>
            <LI>Data File - select the result of the Dimension Reduction run from the file dropdown,     <b>seeded_stream_result.pkl</b></LI>
            <LI>output filename - <b>epg</b> </LI>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [40]:
stream_elasticprincipalgraph_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00399')
stream_elasticprincipalgraph_job_spec = stream_elasticprincipalgraph_task.make_job_spec()
stream_elasticprincipalgraph_job_spec.set_parameter("data.file", "")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.beta", "0.0")
stream_elasticprincipalgraph_job_spec.set_parameter("figure.width", "8")
stream_elasticprincipalgraph_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_elasticprincipalgraph_job_spec.set_parameter("figure.height", "8")
stream_elasticprincipalgraph_job_spec.set_parameter("output.filename", "epg_")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.num.nodes", "50")
stream_elasticprincipalgraph_job_spec.set_parameter("incremental.number.of.nodes", "30")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.trimming.radius", "Inf")
stream_elasticprincipalgraph_job_spec.set_parameter("component.x", "0")
stream_elasticprincipalgraph_job_spec.set_parameter("component.y", "1")
stream_elasticprincipalgraph_job_spec.set_parameter("num.components", "3")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.alpha", "0.02")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.lambda", "0.02")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.mu", "0.1")
stream_elasticprincipalgraph_job_spec.set_parameter("epg.final.energy", "Penalized")
stream_elasticprincipalgraph_job_spec.set_parameter("job.memory", "8 Gb")
stream_elasticprincipalgraph_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_elasticprincipalgraph_job_spec.set_parameter("job.cpuCount", "2")
stream_elasticprincipalgraph_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_elasticprincipalgraph_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00399')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"epg_branches.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Set the <b>"Second image File Url"</b> parameter to <b>"epg_branches_with_cells.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [21]:
genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

Although ElPiGraph is a general approach to construct principal trees (and other topologies), the obtained structures may not optimally describe biologically relevant trajectories or accurately capture pseudotime information based on single-cell data. Therefore, in addition to the described seeding strategy, in STREAM several single-cell specific optimizations were introduced to the core algorithm of ElPiGraph:

<img src="https://github.com/pinellolab/STREAM/raw/master/tutorial/img/Sup_Figure01.png" width="600">


* **Control over-branching**: A regularization parameter α with range (0,1] was introduced to explicitly control the complexity of the resulting graph structure. Larger values of α lower the propensity of ElPiGraph to introduce branching points. An extreme value close to one prevents the creation of new branches not present in the initial seeding structure. Users can control this parameter based on the expected characteristics of noise and dimensionality of the data. By default, α is set 0.02 and this value was used for all the analyses performed.(It is integrated into step `Elastic Principal Graphs`)
* **Prune branches**: The standard elastic principal graph favors harmonicity, i.e., star-shape subgraphs with a central node connected with equally spaced nodes. We have observed that this may lead to trivial branches with few cells. With the pruning grammar rule, STREAM is able to remove branches that are either associated with an excessively small number of cells or that are shorter than a minimal length. This step helps to get rid of spurious and unnecessary branching events that may not reflect real developmental trajectories.(It is implemented in `Prune Graph`)
* **Shift branching nodes**: To minimize the elastic principal graph energy, ElPiGraph balances the reconstruction error, total length of edges, and the graph harmonicity. However, it may happen that the optimal solution places branching nodes to low-density regions with few cells. This can be hard to interpret biologically since these nodes should correspond to branching cell states within the cell population. With the shift branching grammar rule, each branching node is repositioned to the closest area with higher cell density to better match the most plausible region corresponding to the true branching event. (It is not currently supported yet. We will add it in the next release.)
* **Finetune branching nodes**: The standard elastic principal graph procedure well summarizes the principal structure. However, branching sections (i.e., regions close to branching events) may be not described in sufficient details by the obtained curves due to the limited number of nodes used around branching nodes by the global optimization. This grammar rule is able to optimize the space around branching nodes by locally adding a set of nodes in their proximity to better characterize branching events and to improve the pseudotime inference. (It is implemented in `Optimize Structure`)
* **Extend leaf nodes**: The standard elastic principal graph penalizes the total edge length to more robustly capture the main underlying structure. Although this strategy works well when optimizing the graph structure, it may lead to border effects when projecting the data onto the leaf nodes. In fact, edges connecting internal node to leaf nodes rarely extend to the border of their local cloud of points. This is not ideal in the context of pseudotime reconstruction since multiple cells would be mapped to the same leaf node and assigned the same pseudotime. Hence, we extend the principal tree by attaching a new node to each leaf. The location of each node is based on the distribution of points around the corresponding leaf node. This enables the principal graph to better cover terminal cells and infer more accurately their pseudotime. (It is implemented in `Extend Leaf Nodes`)

By default, only **"Finetune branching nodes"** and **"Extend leaf nodes"** are enabled.

<div class="alert alert-info">
<p class="lead"> Instructions - Adjusting the Elastic Principal Graph <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.EPGAdjustFinalGraph module form below<br/>
        <UL>
            <LI>Data File - select the result of the Dimension Reduction run from the file dropdown,     <b>epg_stream_result.pkl</b></LI>
            <LI>output filename - <b>aepg</b> </LI>
            <LI>epg trimming radius - <b>Inf</b> </LI>
            <LI>epg alpha - <b>0.02</b> </LI>
            <LI>epg beta - <b>0.0</b> </LI>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [41]:
stream_epgadjustfinalgraph_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00400')
stream_epgadjustfinalgraph_job_spec = stream_epgadjustfinalgraph_task.make_job_spec()
stream_epgadjustfinalgraph_job_spec.set_parameter("data.file", "")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.beta", "0.0")
stream_epgadjustfinalgraph_job_spec.set_parameter("figure.width", "8")
stream_epgadjustfinalgraph_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_epgadjustfinalgraph_job_spec.set_parameter("figure.height", "8")
stream_epgadjustfinalgraph_job_spec.set_parameter("output.filename", "aepg")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.max.steps", "50")
stream_epgadjustfinalgraph_job_spec.set_parameter("incremental.number.of.nodes", "30")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.trimming.radius", "Inf")
stream_epgadjustfinalgraph_job_spec.set_parameter("component.x", "0")
stream_epgadjustfinalgraph_job_spec.set_parameter("component.y", "1")
stream_epgadjustfinalgraph_job_spec.set_parameter("num.components", "3")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.alpha", "0.02")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.lambda", "0.02")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.mu", "0.1")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.collapse.mode", "PointNumber")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.collapse.parameter", "5")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.extension.mode", "QuantDists")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.extension.parameter", "0.5")
stream_epgadjustfinalgraph_job_spec.set_parameter("epg.final.energy", "Penalized")
stream_epgadjustfinalgraph_job_spec.set_parameter("job.memory", "8 Gb")
stream_epgadjustfinalgraph_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_epgadjustfinalgraph_job_spec.set_parameter("job.cpuCount", "2")
stream_epgadjustfinalgraph_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_epgadjustfinalgraph_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00400')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"aepg_branches.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Set the <b>"Second image File Url"</b> parameter to <b>"aepg_branches_with_cells.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [22]:
genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

## UMAP validation of branch assignment

**This step is helpful only when keeping more than two components in the step of Dimension Reduction.**

Before we proceed with downstream visualizations of cells and gene expression patterns, we will check if the learned structure (trajectories) makes sense. If it does, we will continue the downstream visualization of trajectories.  If not, we would go back to previous steps to modify the parameters used to re-run structure learning steps.

To check the data, we will import the previously calcuated UMAP or tSNE coordinates of cells and color cells by learned branch assignment. If the branch assignment fits well the cellular landscape, the learned structure will be considered reasonable.

<div class="alert alert-info">
<p class="lead"> Instructions - Validation of branch assignment <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.Plot2DVisualization module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>2dCell</b></LI>
            <LI>method - <b>Uniform Manifold Approximation and Projection (UMAP)</b></LI>
            <li>percent neighbor cells - <b>0.1</b></li>
            <li>color by - <b>label</b></li>
            <li>use precomputed - <b>false</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [42]:
stream_plot2dvisualization_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00406')
stream_plot2dvisualization_job_spec = stream_plot2dvisualization_task.make_job_spec()
stream_plot2dvisualization_job_spec.set_parameter("data.file", "")
stream_plot2dvisualization_job_spec.set_parameter("percent.neighbor.cells", "0.1")
stream_plot2dvisualization_job_spec.set_parameter("perplexity", "30")
stream_plot2dvisualization_job_spec.set_parameter("color.by", "label")
stream_plot2dvisualization_job_spec.set_parameter("method", "umap")
stream_plot2dvisualization_job_spec.set_parameter("use.precomputed", "")
stream_plot2dvisualization_job_spec.set_parameter("figure.width", "8")
stream_plot2dvisualization_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_plot2dvisualization_job_spec.set_parameter("figure.height", "8")
stream_plot2dvisualization_job_spec.set_parameter("output.filename", "2dCell")
stream_plot2dvisualization_job_spec.set_parameter("job.memory", "8 Gb")
stream_plot2dvisualization_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_plot2dvisualization_job_spec.set_parameter("job.cpuCount", "2")
stream_plot2dvisualization_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_plot2dvisualization_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00406')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"2dCell_2D_plot.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [23]:

genepattern.GPUIBuilder(displayImage, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

## Visualization (cells)

To facilitate the exploration of the inferred structure, STREAM includes a flat tree plot that intuitively represents trajectories as linear segments on a 2D plane.

The tree structure learned in the 3D space (or higher dimensional space), is first approximated by linear segments (each representing a branch) and mapped to a 2D plane based on a modified version of the force-directed layout Fruchterman-Reingold algorithm. We adjust each edge length in order to preserve the lengths of the branches of the original tree. Finally, using both the pseudotime location on the assigned branch and the distance from it in the MLLE space, we map cells to the obtained tree in the 2D plane. Cells are represented as dots and randomly placed to either side of the assigned branches. Each node in the tree indicates one cell state (cell states are sequentially named S0, S1, ... starting from a randomly selected node) and the resulting structure is called **flat tree plot**.

Starting from the flat tree plot and with a designated root or start node, breadth-first search is used to order and arrange nodes and edges horizontally on a 2D plane. Because we preserve the branch lengths of the original tree, the x-axis represents the distance (namely pseudotime) from the start node along the different branches. Cells are then mapped to the obtained structure, and the display of this structure is called a subway map plot. It is created with the same strategy used for the flat tree plot. 

Although these visualizations capture trajectories and branching points, they are not informative on the density and composition of cell types along pseudotime, a common challenge when modeling large datasets. To solve this problem, we develop a trajectory visualization method called the stream plot. This compact representation summarizes cellular developmental trajectories, user-defined annotations, branching points, cell density, and gene expression patterns.

Starting from the subway map plot, for each cell type (if cell labels are provided), using a sliding window approach, we first calculate the number of cells in each window along a developmental branch. To provide smooth transitions around the branching nodes, in those regions the sliding window spans both parent branch and children branches and then proceeds independently on each branch. Then, the numbers of cells in all sliding windows are normalized based on the length of the longest path in the tree. The vertical layout of different branches is optimized by taking into consideration normalized numbers of cells to make sure there will not be overlap between branches. Based on the normalized sliding window values, we first use linear interpolation to construct a set of supporting points. Then the Savitzky-Golay filter (a smoothing filter able to preserve well the signal and avoid oscillations) is applied to create smooth curves based on the set of supporting points. Finally, the obtained curves polygons (one for each cell type) are assembled to form the stream plot. On the stream plot, the length of each branch is the same as in the subway map plot and represents pseudotime, whereas the width is proportional to the number of cells at a given position.

<div class="alert alert-info">

<p class="lead"> Instructions - Visualization of cell types on the trajectory <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.VisualizeTrajectories module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>viz</b></LI>
            <LI>root - <b>S4</b></LI>
            <li>preference - <b>S1,S2,S3,S4,S5</b></li>
            <li>color by - <b>label</b></li>
            <li>create cell plots - <b>Create Cell Plots</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [32]:
stream_visualizetrajectories_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00401')
stream_visualizetrajectories_job_spec = stream_visualizetrajectories_task.make_job_spec()
stream_visualizetrajectories_job_spec.set_parameter("data.file", "")
stream_visualizetrajectories_job_spec.set_parameter("percentile.dist", "100")
stream_visualizetrajectories_job_spec.set_parameter("figure.width", "8")
stream_visualizetrajectories_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_visualizetrajectories_job_spec.set_parameter("figure.height", "8")
stream_visualizetrajectories_job_spec.set_parameter("output.filename", "viz")
stream_visualizetrajectories_job_spec.set_parameter("root", "S4")
stream_visualizetrajectories_job_spec.set_parameter("preference", "S1,S2,S3,S4,S5")
stream_visualizetrajectories_job_spec.set_parameter("subway.factor", "2.0")
stream_visualizetrajectories_job_spec.set_parameter("color.by", "label")
stream_visualizetrajectories_job_spec.set_parameter("factor.num.win", "10")
stream_visualizetrajectories_job_spec.set_parameter("factor.min.win", "2.0")
stream_visualizetrajectories_job_spec.set_parameter("factor.width", "2.5")
stream_visualizetrajectories_job_spec.set_parameter("flag.log.view", "")
stream_visualizetrajectories_job_spec.set_parameter("factor.zoom.in", "100")
stream_visualizetrajectories_job_spec.set_parameter("create.cell.plots", "-flag_cells ")
stream_visualizetrajectories_job_spec.set_parameter("create.gene.plots", "-flag_genes ")
stream_visualizetrajectories_job_spec.set_parameter("genes", "")
stream_visualizetrajectories_job_spec.set_parameter("job.memory", "8 Gb")
stream_visualizetrajectories_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_visualizetrajectories_job_spec.set_parameter("job.cpuCount", "2")
stream_visualizetrajectories_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_visualizetrajectories_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00401')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"viz_cell_stream_plot.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Set the <b>"Second image File Url"</b> parameter to <b>"viz_cell_subway_map.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [24]:
genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

## Visualization (genes)

On subway map plot, to display gene expression, each cell is colored according to its gene expression (the maximum value in the colormap is set as 90 percentile of gene expression values across all cells).

On stream plot, to display gene expression, we consider, for each sliding window, not only the number of cells but also their average gene expression values smoothed by bicubic interpolation (the maximum value is set as the nintieth percentile of the average gene expression values from all the sliding windows). 

<div class="alert alert-info">
<p class="lead"> Instructions - Visualization of genes on the trajectory <i class="fa fa-info-circle"></i></p>
<ol>
    <li>Set the following parameter values in the STREAM.VisualizeTrajectories module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>viz</b></LI>
            <LI>root - <b>S1</b></LI>
            <li>preference - <b>S3,S4</b></li>
            <li>color by - <b>label</b></li>
            <li>create gene plots - <b>Create Gene Plots</b></li>
            <li>genes - <b>Car2, Gata1, Epx</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [33]:
stream_visualizetrajectories_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00401')
stream_visualizetrajectories_job_spec = stream_visualizetrajectories_task.make_job_spec()
stream_visualizetrajectories_job_spec.set_parameter("data.file", "")
stream_visualizetrajectories_job_spec.set_parameter("percentile.dist", "100")
stream_visualizetrajectories_job_spec.set_parameter("figure.width", "8")
stream_visualizetrajectories_job_spec.set_parameter("figure.legend.num.columns", "3")
stream_visualizetrajectories_job_spec.set_parameter("figure.height", "8")
stream_visualizetrajectories_job_spec.set_parameter("output.filename", "figs")
stream_visualizetrajectories_job_spec.set_parameter("root", "S1")
stream_visualizetrajectories_job_spec.set_parameter("preference", "S3,S4")
stream_visualizetrajectories_job_spec.set_parameter("subway.factor", "2.0")
stream_visualizetrajectories_job_spec.set_parameter("color.by", "label")
stream_visualizetrajectories_job_spec.set_parameter("factor.num.win", "10")
stream_visualizetrajectories_job_spec.set_parameter("factor.min.win", "2.0")
stream_visualizetrajectories_job_spec.set_parameter("factor.width", "2.5")
stream_visualizetrajectories_job_spec.set_parameter("flag.log.view", "")
stream_visualizetrajectories_job_spec.set_parameter("factor.zoom.in", "100")
stream_visualizetrajectories_job_spec.set_parameter("create.cell.plots", "-flag_cells ")
stream_visualizetrajectories_job_spec.set_parameter("create.gene.plots", "-flag_genes ")
stream_visualizetrajectories_job_spec.set_parameter("genes", "Car2, Gata1, Epx")
stream_visualizetrajectories_job_spec.set_parameter("job.memory", "8 Gb")
stream_visualizetrajectories_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_visualizetrajectories_job_spec.set_parameter("job.cpuCount", "2")
stream_visualizetrajectories_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_visualizetrajectories_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00401')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"First image File Url"</b> parameter to <b>"S1/stream_plot_car2.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Set the <b>"Second image File Url"</b> parameter to <b>"S1/subway_map_car2.png"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
        <li>Repeat steps 1-3 for the other 2 genes (epx, gata1).</li>
    </ol>
</div>

In [25]:
genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

# Marker gene detection

## 1.Detect marker genes for each leaf branch

For each gene E we scale the gene expression values to [0,1]. Then we calculate the average gene expressions for all leaf branches. Based on the average expressions, we calculate the Z-scores of all leaf branches. If there is any leaf branch with an absolute Z-score greater than 1.5, then the leaf branch with the highest absolute Z-score value will be picked as the candidate leaf branch. Next, Kruskal–Wallis H-test is computed for all the leaf branches to test if a significant difference of gene expression median value between leaf branches exists. If it is significant (p-value < 0.01), then a post-hoc pairwise Conover’s test is computed for multiple comparisons of mean rank sums test between all leaf branches. If the p-values between the candidate leaf branch and the other leaf branches are all below the specified threshold (0.01), then the gene E will be considered as leaf gene of the candidate leaf branch.

<div class="alert alert-info">
<p class="lead"> Instructions - Detect Marker Genes for each leaf branch <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.DetectLeafGenes module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>leaf</b></LI>
            <LI>root - <b>S4</b></LI>
            <li>preference - <b>S1,S2</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [36]:
stream_detectleafgenes_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00403')
stream_detectleafgenes_job_spec = stream_detectleafgenes_task.make_job_spec()
stream_detectleafgenes_job_spec.set_parameter("data.file", "")
stream_detectleafgenes_job_spec.set_parameter("output.filename", "leaf_<data.file_basename>")
stream_detectleafgenes_job_spec.set_parameter("root", "S4")
stream_detectleafgenes_job_spec.set_parameter("preference", "S2,S1")
stream_detectleafgenes_job_spec.set_parameter("cutoff.zscore", "1.5")
stream_detectleafgenes_job_spec.set_parameter("cutoff.pvalue", "0.01")
stream_detectleafgenes_job_spec.set_parameter("percentile.expr", "95")
stream_detectleafgenes_job_spec.set_parameter("use.precomputed", "-flag_use_precomputed ")
stream_detectleafgenes_job_spec.set_parameter("job.memory", "8 Gb")
stream_detectleafgenes_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_detectleafgenes_job_spec.set_parameter("job.cpuCount", "2")
stream_detectleafgenes_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_detectleafgenes_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00403')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Job File Url"</b> parameter to <b>"leaf_genes/leaf_genesS0_S1.tsv"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
        <li>Adjust the z-score slider to expand/contract the list of displayed genes based on their z-scores.</li>
        <li>Repeat with the other output files to see transition genes for different branches.</li>
    </ol>
</div>

In [26]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
import os
import pandas as pd
HideOutputvar_parameters = {"output_var": {"hide": True}}


showLeafMarkerGenes_parameters = {"Job_File_Url": {"type": "file", "kinds": ["tsv", "TSV"]}, "output_var": {"hide": True}}   


@genepattern.build_ui(description = "Download a tsv gene list output file to the Notebook server and display the genes above the z-score cutoff",
                          parameters=showLeafMarkerGenes_parameters, collapse=False)
def showLeafMarkerGenes(Job_File_Url):
    # first get the result file
        

    # extract job number and file name from url
    job_num = Job_File_Url.split("/")[-3]
    remote_file_name = Job_File_Url.split("/")[-1]

    File_Name = remote_file_name
    if os.path.isfile(File_Name):
        print("File already exists, renaming the old file to " + File_Name+ ".old before downloading.")
        os.rename(File_Name, File_Name + ".old")

    
    print(remote_file_name)
    print(job_num)
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    print("downloading file...")
    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    print("Download complete.")
    
    dataframe = pd.read_csv(File_Name, sep='\t')
    def showTopGenes(z_score_cutoff=1.5):
        # Histogram(Data_File, Plot_Variable, Which_Axis, Lower_Bound, Upper_Bound)
        print(z_score_cutoff)
        pd.set_option('display.max_rows', len(dataframe))
        display(dataframe[dataframe['zscore'] > z_score_cutoff])
        pd.reset_option('display.max_rows')
    
    display(interactive(showTopGenes,  z_score_cutoff=FloatSlider(value=1.7, min=1.5, max=2.0, step=0.01,continuous_update=False)))
    

UIBuilder(collapse=False, description='Download a tsv gene list output file to the Notebook server and display…

## 2. Detect transition gene for each branch

For each branch Bi and for each gene E we first scale the gene expression values to [0,1] for convenience. Then we check if the candidate gene has a reasonable dynamic range considering cells close to the start and end points. To this end, we consider the fold change in average gene expressions of the first 20% and the last 80% of the cells based on the inferred pseudotime. If the difference is greater than a specified threshold (the default log2 fold change value is 0.25), we then calculate Spearman’s rank correlation between inferred pseudotime and gene expression of all the cells along Bi. Genes with Spearman’s correlation coefficient above a specified threshold (0.4 by default) are identified and reported as transition genes.

<div class="alert alert-info">
<p class="lead"> Instructions - Detect Transition Genes for each leaf branch <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.DetectTransitionGenes module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>transition</b></LI>
            <LI>root - <b>S4</b></LI>
            <li>preference - <b>S1,S2</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [38]:
stream_detecttransitiongenes_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00404')
stream_detecttransitiongenes_job_spec = stream_detecttransitiongenes_task.make_job_spec()
stream_detecttransitiongenes_job_spec.set_parameter("data.file", "")
stream_detecttransitiongenes_job_spec.set_parameter("figure.width", "8")
stream_detecttransitiongenes_job_spec.set_parameter("use.precomputed", "")
stream_detecttransitiongenes_job_spec.set_parameter("figure.height", "8")
stream_detecttransitiongenes_job_spec.set_parameter("output.filename", "transition")
stream_detecttransitiongenes_job_spec.set_parameter("num.genes", "15")
stream_detecttransitiongenes_job_spec.set_parameter("root", "S4")
stream_detecttransitiongenes_job_spec.set_parameter("preference", "S2,S1")
stream_detecttransitiongenes_job_spec.set_parameter("percentile.expr", "95")
stream_detecttransitiongenes_job_spec.set_parameter("cutoff.spearman", "0.4")
stream_detecttransitiongenes_job_spec.set_parameter("cutoff.logfc", "0.25")
stream_detecttransitiongenes_job_spec.set_parameter("job.memory", "8 Gb")
stream_detecttransitiongenes_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_detecttransitiongenes_job_spec.set_parameter("job.cpuCount", "2")
stream_detecttransitiongenes_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_detecttransitiongenes_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00404')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Image File Url"</b> parameter to <b>"transition_genes_S0_S1.pdf"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [27]:
genepattern.GPUIBuilder(displayPdf, collapse=False,
                        description=displayPdfDescription,
                        parameters=displayPdfParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

## 3. Detect differentially expressed genes between pairs of branches

For each pair of branches $B_i$ and $B_j$, and for the gene E, the gene expression values across cells from both branches are scaled to the range [0,1]. For gene expression $E_i$ from $B_i$ and gene expression $E_j$ from $B_j$, we first calculate their mean values. Then, we check the fold change between mean values to make sure it is above a specified threshold (the default log2 fold change value is >0.25). Mann–Whitney U test is then used to test whether $E_i$ is greater than $E_j$ or $E_i$ is less than $E_j$. Since the statistic U could be approximated by a normal distribution for large samples, and U depends on specific datasets, we standardize U to Z-score to make it comparable between different datasets. For small samples where this test is underpowered (<20 cells per branch), we report only the fold change to qualitatively evaluate the difference between $E_i$ and $E_j$. Genes with Z-score or fold change greater than the specified threshold (2.0 by default) are considered as differentially expressed genes between two branches. Formally:

\begin{equation*}
z = 1 +  \frac{U-m_{U}}{(\sigma_{U})}
\end{equation*}

Where $m_{U}$, $\sigma_{U}$ are the mean and standard deviation, and

\begin{equation*}
m_{U} = \frac{n_{i}n_{j}}{2}
\end{equation*}

\begin{equation*}
\sigma_{U} = \sqrt{\frac{n_{i}n_{j}}{12}}((n+1)-\sum_{l=1}^k\frac{t^3_l-t_l}{n(n-1})
\end{equation*}

Where $n = n_i + n_j$ $n_i$,$n_j$ are the number of cells in each branch, $t_i$ is the number of cells sharing rank $l$ and $k$ is the number of distinct ranks.

<div class="alert alert-info">

<p class="lead"> Instructions - Detect Differentially expressed Genes between pairs of branches <i class="fa fa-info-circle"></i></p>

<ol>
    <li>Set the following parameter values in the STREAM.DetectDifferentiallyExpressedGenes module form below<br/>
        <UL>
            <LI>Data File - select the result of the EPGAdjustFinalGraph run from the file dropdown, <b>aepg_stream_result.pkl</b></LI>
            <LI>output filename - <b>de</b></LI>
            <LI>root - <b>S4</b></LI>
            <li>preference - <b>S2,S1</b></li>
            <li>All other parameters should be defaults or False.</li>
        </UL>
    </li>
    <li>Click the Run Button</li>
</ol>
</div>

In [39]:
stream_detectdifferentiallyexpressedgenes_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00402')
stream_detectdifferentiallyexpressedgenes_job_spec = stream_detectdifferentiallyexpressedgenes_task.make_job_spec()
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("data.file", "")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("figure.width", "8")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("flag.use.precomputed", "")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("figure.height", "8")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("output.filename", "<data.file_basename>")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("num.genes", "15")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("root", "S4")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("preference", "S2,S1")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("cutoff.zscore", "1.5")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("percentile.expr", "95")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("cutoff.logfc", "0.25")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("job.memory", "8 Gb")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("job.queue", "gp-new-beta")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("job.cpuCount", "2")
stream_detectdifferentiallyexpressedgenes_job_spec.set_parameter("job.walltime", "1:00:00")
genepattern.display(stream_detectdifferentiallyexpressedgenes_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00402')

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Set the <b>"Image File Url"</b> parameter to <b>"de_genes_S0_S2 and S0_S1.pdf"</b> from the job result above.  This can be selected from the drop-down selector for the parameter.</li>
        <li>Click 'Run'.</li>
    </ol>
</div>

In [28]:
genepattern.GPUIBuilder(displayPdf, collapse=False,
                        description=displayPdfDescription,
                        parameters=displayPdfParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

<h1 id="Conclusion" data-toc-modified-id="Conclusion-2.1"><a id="Conclusion-2.1" class="toc-mod-link"></a>Conclusion</h1>
<p>For more information about the STREAM Trajectory Inference method or to cite its use, please refer to the paper, "<a href="https://www.nature.com/articles/s41467-019-09670-4">Single-cell trajectories reconstruction, exploration and mapping of omics data with STREAM</a>", Nature Communications, volume 10, Article number: 1903 (2019)</p>
<h2 id="References" data-toc-modified-id="References-2.1.1"><a id="References-2.1.1" class="toc-mod-link"></a>References</h2>
<p><a href="https://www.nature.com/articles/s41467-019-09670-4">H Chen, L Albergante, JY Hsu, CA Lareau, GL Bosco, J Guan, S Zhou, AN Gorban, DE Bauer, MJ Aryee, DM Langenau, A Zinovyev, JD Buenrostro, GC Yuan, L Pinello. Single-cell trajectories reconstruction, exploration and mapping of omics data with STREAM</a>. Nature Communications, volume 10, Article number: 1903 (2019).</p>
<p>&nbsp;</p>
<p><a href="https://github.com/pinellolab/STREAM" target="_blank" rel="noopener">https://github.com/pinellolab/STREAM</a></p>
<p>&nbsp;</p>